In [1]:
import gzip
import os
from chesslm.lichess import LichessPGNEntry

with gzip.open(os.path.expanduser("~/Data/standard_rated_2013-01_filtered.jsonl.gz"), "rb") as f:
    entries = [LichessPGNEntry.model_validate_json(line) for line in f.readlines()]

In [2]:
import random

sample: LichessPGNEntry = random.sample(entries, 1)[0]
display(sample.site)
print(sample.plain_sequence)

'https://lichess.org/fgfzhz1f'

1. d4 Nf6 2. Nc3 e6 3. e4 Be7 4. e5 Nd5 5. Nxd5 exd5 6. Bd3 O-O 7. Nf3 d6 8. exd6 Bxd6 9. O-O Bg4 10. Be2 Nd7 11. Qd3 Nf6 12. Bg5 c6 13. Bxf6 Qxf6 14. h3 Bd7 15. Nh2 Rae8 16. Bg4 Bxh2+ 17. Kxh2 Bxg4 18. hxg4 Qf4+ 19. Qg3 Qxd4 20. c3 Qd2 21. Rab1 Re2 22. Qc7 Rxf2 23. Rxf2 Qxf2 24. Qxb7 Qf4+ 25. Kh3 h5 26. gxh5 Qf5+ 27. g4 Qxb1 28. Qxc6 Qxb2 29. Qxd5 Qxc3+ 30. Kh4 Qf6+ 31. Kh3 Rd8 32. Qb3 Qd4 33. Kh4 Qd3 34. Qa4 Rd4 35. Qxa7 Qe4 36. Qb8+ Kh7 37. Qg3 Rd3 38. Qg1 Qe7+ 39. g5 Qe4+ 40. Qg4 Qe1+ 41. Qg3 Qxg3# 0-1


In [15]:
vocabulary = [
    # general
    " ",
    *map(chr, range(48, 58)),  # 0-9
    *map(chr, range(97, 105)),  # a-h

    # pieces
    "B", "K", "N", "Q", "R"

    # start of turn
    ".",

    # castling
    "O-O", "O-O-O",

    # kill / check(mate) / endgame
    "x", "+", "#", "0-1", "1-0",
]

In [18]:
ord("h")

104